In [1]:
import numpy as np 
from data_and_models import dynamical_model, generate_data, handle_NNs
import pickle 
import matplotlib.pyplot as plt 
from pathlib import Path
import os
import time 

import tensorflow as tf 
from kerastuner.tuners import RandomSearch

In [2]:
#x = [T_m, T_c]
#u = [Q_in, T_env]
#theta = [C_m, C_c, hA_m2c, hA_m2env, hA_eff, mda]

chp_air = 1000 
Q_HE = lambda x, u, theta: theta[5]*chp_air*(1-np.exp(-theta[4]/(theta[5]*chp_air)))*(x[1]-u[1])
dTm_dt = lambda x, u, theta: (u[0]-theta[2]*(x[0]-x[1])-theta[3]*(x[0]-u[1]))/theta[0]
dTc_dt = lambda x, u, theta: (theta[2]*(x[0]-x[1])-Q_HE(x,u,theta))/theta[1]

dxdt = lambda x,u,theta: np.array([dTm_dt(x,u,theta), dTc_dt(x,u,theta)])
f_x = lambda x,u,theta: np.array([Q_HE(x,u,theta)])

x0_range = [(20, 60), (20, 60)]
u_range = [(0, 30), (20, 30)]
theta_range = [(100, 800), (40, 400), (5,20), (0.1,1), (1,5), (0.3, 1.2)]

model = dynamical_model(dxdt, f_x)
model.setup(0, 60, 600, 20, x0_range, u_range, theta_range)

In [3]:
data_model = generate_data(model) 
#X_NN, y_NN = data_model.load_data("test_x_data.npy", "test_y_data.npy") #could also generate data 
X_NN, y_NN = data_model.solve_N(2e6, "X_data.npy", "y_data.npy")
X_train, X_test, y_train, y_test, X_NN_scaled, y_NN_scaled = data_model.prepare_data(X_NN, y_NN)

handler = handle_NNs(len(X_train[0,:]), len(y_train[0,:]))
tuner = RandomSearch(
    handler.build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    project_name="Param_est_Hyperparams", 
    directory=os.path.normpath('C:/'))

tuner.search_space_summary()

In [14]:
t1 = time.time() 
tuner.search(X_train, y_train, epochs=3, validation_data=(X_test, y_test), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)],verbose=2)
print()

Epoch 1/5


215/282 [=====================>........] - ETA: 0s - loss: 0.3666 - accuracy: 0.12 - ETA: 11s - loss: 0.2972 - accuracy: 0.156 - ETA: 18s - loss: 0.2444 - accuracy: 0.156 - ETA: 18s - loss: 0.2127 - accuracy: 0.156 - ETA: 20s - loss: 0.1998 - accuracy: 0.156 - ETA: 19s - loss: 0.1872 - accuracy: 0.151 - ETA: 21s - loss: 0.1750 - accuracy: 0.165 - ETA: 20s - loss: 0.1649 - accuracy: 0.156 - ETA: 20s - loss: 0.1594 - accuracy: 0.149 - ETA: 20s - loss: 0.1550 - accuracy: 0.159 - ETA: 19s - loss: 0.1448 - accuracy: 0.169 - ETA: 19s - loss: 0.1401 - accuracy: 0.173 - ETA: 20s - loss: 0.1361 - accuracy: 0.171 - ETA: 20s - loss: 0.1328 - accuracy: 0.168 - ETA: 20s - loss: 0.1302 - accuracy: 0.162 - ETA: 19s - loss: 0.1283 - accuracy: 0.161 - ETA: 19s - loss: 0.1266 - accuracy: 0.163 - ETA: 19s - loss: 0.1258 - accuracy: 0.156 - ETA: 19s - loss: 0.1246 - accuracy: 0.154 - ETA: 19s - loss: 0.1232 - accuracy: 0.153 - ETA: 19s - loss: 0.1217 - accuracy: 0.154 - ETA: 19s - loss: 0.1206 - accuracy:

282/282 [==============================] - ETA: 4s - loss: 0.0921 - accuracy: 0.18 - ETA: 4s - loss: 0.0920 - accuracy: 0.18 - ETA: 4s - loss: 0.0921 - accuracy: 0.18 - ETA: 4s - loss: 0.0921 - accuracy: 0.18 - ETA: 4s - loss: 0.0921 - accuracy: 0.18 - ETA: 4s - loss: 0.0921 - accuracy: 0.18 - ETA: 4s - loss: 0.0921 - accuracy: 0.18 - ETA: 4s - loss: 0.0920 - accuracy: 0.18 - ETA: 4s - loss: 0.0920 - accuracy: 0.18 - ETA: 4s - loss: 0.0919 - accuracy: 0.18 - ETA: 4s - loss: 0.0919 - accuracy: 0.18 - ETA: 3s - loss: 0.0919 - accuracy: 0.18 - ETA: 3s - loss: 0.0918 - accuracy: 0.18 - ETA: 3s - loss: 0.0918 - accuracy: 0.18 - ETA: 3s - loss: 0.0917 - accuracy: 0.18 - ETA: 3s - loss: 0.0917 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy: 0.18 - ETA: 3s - loss: 0.0916 - accuracy

196/282 [===================>..........] - ETA: 0s - loss: 0.0914 - accuracy: 0.18 - ETA: 10s - loss: 0.0867 - accuracy: 0.125 - ETA: 14s - loss: 0.0888 - accuracy: 0.104 - ETA: 17s - loss: 0.0891 - accuracy: 0.125 - ETA: 17s - loss: 0.0886 - accuracy: 0.143 - ETA: 17s - loss: 0.0882 - accuracy: 0.151 - ETA: 16s - loss: 0.0873 - accuracy: 0.156 - ETA: 17s - loss: 0.0881 - accuracy: 0.152 - ETA: 18s - loss: 0.0881 - accuracy: 0.156 - ETA: 18s - loss: 0.0887 - accuracy: 0.156 - ETA: 18s - loss: 0.0886 - accuracy: 0.150 - ETA: 22s - loss: 0.0881 - accuracy: 0.164 - ETA: 22s - loss: 0.0877 - accuracy: 0.163 - ETA: 21s - loss: 0.0875 - accuracy: 0.166 - ETA: 20s - loss: 0.0867 - accuracy: 0.169 - ETA: 20s - loss: 0.0868 - accuracy: 0.173 - ETA: 19s - loss: 0.0874 - accuracy: 0.172 - ETA: 19s - loss: 0.0880 - accuracy: 0.175 - ETA: 19s - loss: 0.0881 - accuracy: 0.176 - ETA: 19s - loss: 0.0881 - accuracy: 0.178 - ETA: 20s - loss: 0.0878 - accuracy: 0.182 - ETA: 20s - loss: 0.0877 - accuracy:

282/282 [==============================] - ETA: 7s - loss: 0.0863 - accuracy: 0.20 - ETA: 7s - loss: 0.0863 - accuracy: 0.19 - ETA: 7s - loss: 0.0863 - accuracy: 0.19 - ETA: 6s - loss: 0.0863 - accuracy: 0.19 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 6s - loss: 0.0863 - accuracy: 0.19 - ETA: 6s - loss: 0.0863 - accuracy: 0.19 - ETA: 6s - loss: 0.0863 - accuracy: 0.20 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 6s - loss: 0.0862 - accuracy: 0.19 - ETA: 5s - loss: 0.0862 - accuracy: 0.19 - ETA: 5s - loss: 0.0861 - accuracy: 0.20 - ETA: 5s - loss: 0.0861 - accuracy: 0.20 - ETA: 5s - loss: 0.0861 - accuracy: 0.20 - ETA: 5s - loss: 0.0861 - accuracy: 0.20 - ETA: 5s - loss: 0.0860 - accuracy: 0.20 - ETA: 5s - loss: 0.0861 - accuracy: 0.20 - ETA: 5s - loss: 0.0860 - accuracy: 0.19 - ETA: 5s - loss: 0.0860 - accuracy

...

186/282 [==================>...........] - ETA: 0s - loss: 0.0853 - accuracy: 0.31 - ETA: 33s - loss: 0.0882 - accuracy: 0.234 - ETA: 44s - loss: 0.0886 - accuracy: 0.177 - ETA: 48s - loss: 0.0889 - accuracy: 0.156 - ETA: 50s - loss: 0.0891 - accuracy: 0.150 - ETA: 53s - loss: 0.0892 - accuracy: 0.145 - ETA: 54s - loss: 0.0890 - accuracy: 0.133 - ETA: 54s - loss: 0.0905 - accuracy: 0.132 - ETA: 54s - loss: 0.0897 - accuracy: 0.135 - ETA: 54s - loss: 0.0893 - accuracy: 0.140 - ETA: 54s - loss: 0.0898 - accuracy: 0.142 - ETA: 55s - loss: 0.0897 - accuracy: 0.135 - ETA: 56s - loss: 0.0899 - accuracy: 0.132 - ETA: 58s - loss: 0.0901 - accuracy: 0.133 - ETA: 58s - loss: 0.0904 - accuracy: 0.133 - ETA: 59s - loss: 0.0899 - accuracy: 0.130 - ETA: 59s - loss: 0.0901 - accuracy: 0.137 - ETA: 1:00 - loss: 0.0903 - accuracy: 0.14 - ETA: 1:00 - loss: 0.0904 - accuracy: 0.14 - ETA: 1:01 - loss: 0.0907 - accuracy: 0.14 - ETA: 1:01 - loss: 0.0909 - accuracy: 0.14 - ETA: 1:01 - loss: 0.0906 - accuracy

282/282 [==============================] - ETA: 25s - loss: 0.0894 - accuracy: 0.172 - ETA: 24s - loss: 0.0894 - accuracy: 0.173 - ETA: 24s - loss: 0.0894 - accuracy: 0.173 - ETA: 24s - loss: 0.0894 - accuracy: 0.174 - ETA: 24s - loss: 0.0894 - accuracy: 0.174 - ETA: 23s - loss: 0.0893 - accuracy: 0.174 - ETA: 23s - loss: 0.0893 - accuracy: 0.174 - ETA: 23s - loss: 0.0893 - accuracy: 0.173 - ETA: 22s - loss: 0.0893 - accuracy: 0.173 - ETA: 22s - loss: 0.0893 - accuracy: 0.173 - ETA: 22s - loss: 0.0894 - accuracy: 0.173 - ETA: 22s - loss: 0.0894 - accuracy: 0.172 - ETA: 21s - loss: 0.0895 - accuracy: 0.172 - ETA: 21s - loss: 0.0895 - accuracy: 0.172 - ETA: 21s - loss: 0.0895 - accuracy: 0.172 - ETA: 21s - loss: 0.0896 - accuracy: 0.172 - ETA: 20s - loss: 0.0896 - accuracy: 0.171 - ETA: 20s - loss: 0.0896 - accuracy: 0.171 - ETA: 20s - loss: 0.0896 - accuracy: 0.171 - ETA: 20s - loss: 0.0896 - accuracy: 0.171 - ETA: 19s - loss: 0.0896 - accuracy: 0.170 - ETA: 19s - loss: 0.0896 - accurac

186/282 [==================>...........] - ETA: 0s - loss: 0.0891 - accuracy: 0.21 - ETA: 32s - loss: 0.0877 - accuracy: 0.171 - ETA: 42s - loss: 0.0882 - accuracy: 0.187 - ETA: 48s - loss: 0.0870 - accuracy: 0.195 - ETA: 51s - loss: 0.0884 - accuracy: 0.181 - ETA: 53s - loss: 0.0884 - accuracy: 0.166 - ETA: 54s - loss: 0.0879 - accuracy: 0.156 - ETA: 56s - loss: 0.0872 - accuracy: 0.156 - ETA: 56s - loss: 0.0869 - accuracy: 0.156 - ETA: 57s - loss: 0.0866 - accuracy: 0.162 - ETA: 59s - loss: 0.0877 - accuracy: 0.170 - ETA: 1:01 - loss: 0.0879 - accuracy: 0.17 - ETA: 1:02 - loss: 0.0878 - accuracy: 0.17 - ETA: 1:02 - loss: 0.0881 - accuracy: 0.16 - ETA: 1:02 - loss: 0.0885 - accuracy: 0.16 - ETA: 1:03 - loss: 0.0879 - accuracy: 0.16 - ETA: 1:03 - loss: 0.0884 - accuracy: 0.16 - ETA: 1:03 - loss: 0.0888 - accuracy: 0.16 - ETA: 1:03 - loss: 0.0891 - accuracy: 0.16 - ETA: 1:04 - loss: 0.0893 - accuracy: 0.16 - ETA: 1:03 - loss: 0.0895 - accuracy: 0.16 - ETA: 1:03 - loss: 0.0894 - accuracy

282/282 [==============================] - ETA: 25s - loss: 0.0899 - accuracy: 0.169 - ETA: 24s - loss: 0.0898 - accuracy: 0.169 - ETA: 24s - loss: 0.0898 - accuracy: 0.169 - ETA: 24s - loss: 0.0898 - accuracy: 0.170 - ETA: 23s - loss: 0.0897 - accuracy: 0.170 - ETA: 23s - loss: 0.0897 - accuracy: 0.170 - ETA: 23s - loss: 0.0897 - accuracy: 0.171 - ETA: 23s - loss: 0.0897 - accuracy: 0.171 - ETA: 22s - loss: 0.0897 - accuracy: 0.171 - ETA: 22s - loss: 0.0896 - accuracy: 0.170 - ETA: 22s - loss: 0.0896 - accuracy: 0.170 - ETA: 22s - loss: 0.0896 - accuracy: 0.171 - ETA: 21s - loss: 0.0896 - accuracy: 0.171 - ETA: 21s - loss: 0.0896 - accuracy: 0.171 - ETA: 21s - loss: 0.0896 - accuracy: 0.171 - ETA: 21s - loss: 0.0896 - accuracy: 0.171 - ETA: 20s - loss: 0.0896 - accuracy: 0.171 - ETA: 20s - loss: 0.0896 - accuracy: 0.171 - ETA: 20s - loss: 0.0896 - accuracy: 0.170 - ETA: 20s - loss: 0.0896 - accuracy: 0.170 - ETA: 19s - loss: 0.0896 - accuracy: 0.170 - ETA: 19s - loss: 0.0896 - accurac

INFO:tensorflow:Oracle triggered exit


In [16]:
best_model = tuner.get_best_models(1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [19]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 1024)              7168      
_________________________________________________________________
Dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
Layer_2 (Dense)              (None, 769)               788225    
_________________________________________________________________
Dropout_2 (Dropout)          (None, 769)               0         
_________________________________________________________________
Layer_3 (Dense)              (None, 515)               396550    
_________________________________________________________________
Dropout_3 (Dropout)          (None, 515)               0         
_________________________________________________________________
Layer_4 (Dense)              (None, 260)               1

In [25]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('n_starting_nodes')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 1024 and the optimal learning rate for the optimizer
is 0.001.



In [32]:
tuner.get_best_models(10)[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 1024)              7168      
_________________________________________________________________
Dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
Layer_2 (Dense)              (None, 769)               788225    
_________________________________________________________________
Dropout_2 (Dropout)          (None, 769)               0         
_________________________________________________________________
Layer_3 (Dense)              (None, 515)               396550    
_________________________________________________________________
Dropout_3 (Dropout)          (None, 515)               0         
_________________________________________________________________
Layer_4 (Dense)              (None, 260)               1